In [23]:
import time
import torch

from rcpsp_inst import load_dataset
from net import Net
from aco import ACO_RCPSP

lr = 1e-3
EPS = 1e-10
ALPHA = 0.05
T=20
device = 'cpu'
acoparam = dict(elitist = True, min_max = True)

In [24]:
@torch.no_grad()
def infer_instance(model, rcpsp, n_ants, t_aco_diff):
    if model:
        model.eval()
        pyg_data = rcpsp.to_pyg_data()
        phe_vec, heu_vec = model(pyg_data, require_phe=True, require_heu=True)
        heu_mat = model.reshape(pyg_data, heu_vec) + EPS
    
        aco = ACO_RCPSP(
            rcpsp,
            n_ants=n_ants,
            pheromone=None,
            heuristic=heu_mat,
            device=device,
            **acoparam
        )
    
    else:
        aco = ACO_RCPSP(
            rcpsp,
            n_ants=n_ants,
            device=device,
            **acoparam
        )
        
    results = torch.zeros(size=(len(t_aco_diff),), device=device)
    for i, t in enumerate(t_aco_diff):
        best_cost = aco.run(t).cost
        results[i] = best_cost
    return results
        
    
@torch.no_grad()
def test(dataset, model, n_ants, t_aco):
    _t_aco = [0] + t_aco
    t_aco_diff = [_t_aco[i+1]-_t_aco[i] for i in range(len(_t_aco)-1)]
    sum_results = torch.zeros(size=(len(t_aco_diff),), device=device)
    start = time.time()
    for instance in dataset:
        results = infer_instance(model, instance, n_ants, t_aco_diff)
        sum_results += results
    end = time.time()
    
    return sum_results / len(dataset), end-start

### Test on j30

In [25]:
n_ants = 20
n_node = 30
t_aco = [1, 10, 20, 30, 40, 50, 100]
_, test_list = load_dataset(f"../data/rcpsp/j{n_node}rcp")

In [26]:
# DeepACO
net_tsp = Net().to(device)
net_tsp.load_state_dict(torch.load(f'../pretrained/rcpsp/rcpsp{n_node}-5.pt', map_location=device))
avg_aco_best, duration = test(test_list, net_tsp, n_ants, t_aco)
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    print("T={}, average cost is {}.".format(t, avg_aco_best[i]))

total duration:  132.94564294815063
T=1, average cost is 61.06999969482422.
T=10, average cost is 59.02000045776367.
T=20, average cost is 58.599998474121094.
T=30, average cost is 58.5.
T=40, average cost is 58.15999984741211.
T=50, average cost is 58.02000045776367.
T=100, average cost is 57.75.


In [27]:
# ACO
avg_aco_best, duration = test(test_list, None, n_ants, t_aco)
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    print("T={}, average cost is {}.".format(t, avg_aco_best[i]))

total duration:  132.5256588459015
T=1, average cost is 63.880001068115234.
T=10, average cost is 59.959999084472656.
T=20, average cost is 59.599998474121094.
T=30, average cost is 59.25.
T=40, average cost is 59.060001373291016.
T=50, average cost is 58.970001220703125.
T=100, average cost is 58.77000045776367.


### Test on j60

In [28]:
n_ants = 20
n_node = 60
t_aco = [1, 10, 20, 30, 40, 50, 100]
_, test_list = load_dataset(f"../data/rcpsp/j{n_node}rcp")

In [31]:
# DeepACO
net_tsp = Net().to(device)
net_tsp.load_state_dict(torch.load(f'../pretrained/rcpsp/rcpsp{n_node}-5.pt', map_location=device))
avg_aco_best, duration = test(test_list, net_tsp, n_ants, t_aco)
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    print("T={}, average cost is {}.".format(t, avg_aco_best[i]))

total duration:  261.567898273468
T=1, average cost is 95.30000305175781.
T=10, average cost is 90.62999725341797.
T=20, average cost is 89.33999633789062.
T=30, average cost is 88.69000244140625.
T=40, average cost is 88.41000366210938.
T=50, average cost is 88.0999984741211.
T=100, average cost is 87.16999816894531.


In [32]:
# ACO
avg_aco_best, duration = test(test_list, None, n_ants, t_aco)
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    print("T={}, average cost is {}.".format(t, avg_aco_best[i]))

total duration:  261.57174706459045
T=1, average cost is 99.33999633789062.
T=10, average cost is 93.80000305175781.
T=20, average cost is 92.80999755859375.
T=30, average cost is 92.16999816894531.
T=40, average cost is 91.86000061035156.
T=50, average cost is 91.66000366210938.
T=100, average cost is 90.77999877929688.


### Test on j120

In [33]:
n_ants = 20
n_node = 120
t_aco = [1, 10, 20, 30, 40, 50, 100]
_, test_list = load_dataset(f"../data/rcpsp/j{n_node}rcp")

In [34]:
# DeepACO
net_tsp = Net().to(device)
net_tsp.load_state_dict(torch.load(f'../pretrained/rcpsp/rcpsp{n_node}-5.pt', map_location=device))
avg_aco_best, duration = test(test_list, net_tsp, n_ants, t_aco)
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    print("T={}, average cost is {}.".format(t, avg_aco_best[i]))    

total duration:  530.4030299186707
T=1, average cost is 184.35000610351562.
T=10, average cost is 177.24000549316406.
T=20, average cost is 175.5.
T=30, average cost is 174.55999755859375.
T=40, average cost is 174.0500030517578.
T=50, average cost is 173.5800018310547.
T=100, average cost is 172.0800018310547.


In [35]:
# ACO
avg_aco_best, duration = test(test_list, None, n_ants, t_aco)
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    print("T={}, average cost is {}.".format(t, avg_aco_best[i]))

total duration:  530.2516784667969
T=1, average cost is 193.77999877929688.
T=10, average cost is 185.6199951171875.
T=20, average cost is 183.3800048828125.
T=30, average cost is 182.52999877929688.
T=40, average cost is 181.75.
T=50, average cost is 181.2899932861328.
T=100, average cost is 179.64999389648438.
